In [1]:
import os
import sys
os.environ['SPARK_HOME']='/usr/lib/spark'
os.environ['PYLIB']=os.environ['SPARK_HOME']+'/python/lib'
sys.path.insert(0,os.environ['PYLIB']+'/py4j-0.10.7-src.zip')
sys.path.insert(1,os.environ['PYLIB']+'/pyspark.zip')

In [2]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

In [5]:
spark = SparkSession.builder.appName('TestHive') \
.config('spark.warehouse.dir','/apps/hive/warehouse') \
.config('spark.driver.extraClassPath', 
        '/home/cloudera/.m2/repository/mysql/mysql-connector-java/5.1.44/mysql-connector-java-5.1.44.jar') \
.config('spark.executor.extraClassPath', 
        '/home/cloudera/.m2/repository/mysql/mysql-connector-java/5.1.44/mysql-connector-java-5.1.44.jar') \
.enableHiveSupport().getOrCreate()

In [6]:
# Having set the driver and driver options we should have spark representing spark session 
# available straight away
spark.version

'2.3.0'

In [7]:
sc = spark.sparkContext

In [42]:
prop = {'user': 'root', 'password': 'cloudera', 'driver': 'com.mysql.jdbc.Driver'}
url = 'jdbc:mysql://localhost:3306/testdb'
mrdf = spark.read.format('jdbc').option('url', url).option('dbtable', 'simptable') \
.option('user', 'root').option('password', 'cloudera') \
.option('driver', 'com.mysql.jdbc.Driver').load()
mrdf.show()

+--------+--------+---+
|   fname|   lname|age|
+--------+--------+---+
|    anil|  kapoor| 54|
|  sanjay|    dutt| 56|
|   arjun|  kappor| 28|
|  ranbir|  kappor| 31|
|  deepka|padukone| 29|
|shahrukh|    khan| 51|
| hrithik|  roshan| 37|
+--------+--------+---+



In [28]:
mrdf.printSchema()

root
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)
 |-- age: integer (nullable = true)



In [30]:
mrdf.groupBy('lname').count().show()

+--------+-----+
|   lname|count|
+--------+-----+
|padukone|    1|
|  roshan|    1|
|  kappor|    2|
|    khan|    1|
|  kapoor|    1|
|    dutt|    1|
+--------+-----+



In [27]:
mrdf_age_lim = spark.read.format('jdbc').option('url', url).option('dbtable', 'simptable') \
.option('user', 'root').option('password', 'cloudera') \
.option('driver', 'com.mysql.jdbc.Driver') \
.option('partitionColumn', 'age').option('lowerBound', 10).option('upperBound', 30) \
.option('numPartitions', 2).load()
mrdf_age_lim.show()

+--------+--------+---+
|   fname|   lname|age|
+--------+--------+---+
|    anil|  kapoor| 54|
|  sanjay|    dutt| 56|
|   arjun|  kappor| 28|
|  ranbir|  kappor| 31|
|  deepka|padukone| 29|
|shahrukh|    khan| 51|
| hrithik|  roshan| 37|
+--------+--------+---+



In [46]:
spark.read.format('jdbc').option('url', url).option('dbtable', 'nstable') \
.option('user', 'root').option('password', 'cloudera') \
.option('driver', 'com.mysql.jdbc.Driver').load().count()

63

In [45]:
mrdf.write.jdbc(url=url, mode='append', table='nstable', properties=prop)